# VAE分析 - 基于 Faces 数据集
## 引入

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm

from utils import load_model, ImageLabelLoader
from models import VariationalAutoEncoder

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

## 环境参数

In [ ]:
# run params
SECTION = 'vae'
RUN_ID = '0001'
DATA_NAME = 'faces'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

DATA_FOLDER = './data/celeb/'
IMAGE_FOLDER = './data/celeb/img_align_celeba/'

## 加载数据

In [ ]:
INPUT_DIM = (128,128,3)

att = pd.read_csv(os.path.join(DATA_FOLDER, "list_attr_celeba.csv"))
att.head()

image_loader = ImageLabelLoader(IMAGE_FOLDER, INPUT_DIM[:2])

## 加载模型及参数

In [ ]:
vae = load_model(VariationalAutoEncoder, RUN_FOLDER)

## 重构面部图像

In [ ]:
n_to_show = 10
data_flow_generic = image_loader.build(att, n_to_show)

example_batch = next(data_flow_generic)
example_images = example_batch[0]

z_points = vae.encoder.predict(example_images)
reconst_images = vae.decoder.predict(z_points)

fig = plt.figure(figsize=(15, 3))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i in range(n_to_show):
    img = example_images[i].squeeze()
    sub = fig.add_subplot(2, n_to_show, i+1)
    sub.axis("off")
    sub.imshow(img)

for i in range(n_to_show):
    img = reconst_images[i].squeeze()
    sub = fig.add_subplot(2, n_to_show, i+n_to_show+1)
    sub.axis("off")
    sub.imshow(img)

## 隐空间特征点的分布

In [ ]:
z_test = vae.encoder.predict_generator(data_flow_generic, steps=20, verbose=1)

x = np.linspace(-3, 3, 100)

fig = plt.figure(figsize=(25, 10))
fig.subplots_adjust(hspace=0.4, wspace=0.4)

for i in range(50):
    ax = fig.add_subplot(5, 10, i+1)
    ax.hist(z_test[:, i], density=True, bins=20)
    ax.axis("off")
    ax.text(0.5, -0.35, str(i), fontsize=10, ha="center", transform=ax.transAxes)
    ax.plot(x, norm.pdf(x))

plt.show()

## 生成新面孔

In [ ]:
n_to_show = 30

znew = np.random.normal(size=(n_to_show, vae.z_dim))

reconst = vae.decoder.predict(znew)

fig = plt.figure(figsize=(18, 5))
fig.subplots_adjust(hspace=0.4, wspace=0.4)
for i in range(n_to_show):
    ax = fig.add_subplot(3, 10, i+1)
    ax.imshow(reconst[i, :, :, :])
    ax.axis("off")
plt.show()

## 隐空间算数

In [ ]:
def get_vector_from_label(label, batch_size):
    data_flow_label = image_loader.build(att, batch_size=batch_size, label=label)

    origin = np.zeros(shape=(vae.z_dim), dtype="float32")
    current_sum_POS = np.zeros(shape=(vae.z_dim), dtype="float32")
    current_n_POS = 0
    current_mean_POS = np.zeros(shape=(vae.z_dim), dtype="float32")

    current_sum_NEG = np.zeros(shape=(vae.z_dim), dtype="float32")
    current_n_NEG = 0
    current_mean_NEG = np.zeros(shape=(vae.z_dim), dtype="float32")

    current_vector = np.zeros(shape=(vae.z_dim), dtype="float32")
    current_dist = 0

    print("label: " + label)
    print("images : POS move : NEG move : distance : 𝛥 distance")
    while current_n_POS < 10000:
        batch = next(data_flow_label)
        im = batch[0]
        attribute = batch[1]

        z = vae.encoder.predict(im)

        z_POS = z[attribute==1]
        z_NEG = z[attribute==-1]

        if len(z_POS) > 0:
            current_sum_POS = current_sum_POS + np.sum(z_POS, axis=0)
            current_n_POS += len(z_POS)
            new_mean_POS = current_sum_POS / current_n_POS
            movement_POS = np.linalg.norm(new_mean_POS - current_mean_POS)
        
        if len(z_NEG) > 0:
            current_sum_NEG = current_sum_NEG + np.sum(z_NEG, axis=0)
            current_n_NEG += len(z_NEG)
            new_mean_NEG = current_sum_NEG / current_n_NEG
            movement_NEG = np.linalg.norm(new_mean_NEG - current_mean_NEG)
        
        current_vector = new_mean_POS - new_mean_NEG
        new_dist = np.linalg.norm(current_vector)
        dist_change = new_dist - current_dist

        print(str(current_n_POS)
              + "\t: " + str(np.round(movement_POS, 3))
              + "\t: " + str(np.round(movement_NEG, 3))
              + "\t: " + str(np.round(new_dist, 3))
              + "\t: " + str(np.round(dist_change, 3)))
        
        current_mean_POS = np.copy(new_mean_POS)
        current_mean_NEG = np.copy(new_mean_NEG)
        current_dist = np.copy(new_dist)

        if np.sum([movement_POS, movement_NEG]) < 0.08:
            current_vector = current_vector / current_dist
            print("Found the " + label + " vector")
            break
    
    return current_vector

In [ ]:
def add_vector_to_images(feature_vec):
    n_to_show = 5
    factors = [-4, -3, -2, -1, 0, 1, 2, 3, 4]

    example_batch = next(data_flow_generic)
    example_images = example_batch[0]

    z_points = vae.encoder.predict(example_images)

    fig = plt.figure(figsize=(18, 10))
    counter = 1
    for i in range(n_to_show):
        img = example_images[i].squeeze()
        sub = fig.add_subplot(n_to_show, len(factors)+1, counter)
        sub.axis("off")
        sub.imshow(img)

        counter += 1

        for factor in factors:
            changed_z_point = z_points[i] + feature_vec * factor
            changed_image = vae.decoder.predict(np.array([changed_z_point]))[0]

            img = changed_image.squeeze()
            sub = fig.add_subplot(n_to_show, len(factors)+1, counter)
            sub.axis("off")
            sub.imshow(img)

            counter += 1
    plt.show()

In [ ]:
BATCH_SIZE = 500
attractive_vec = get_vector_from_label('Attractive', BATCH_SIZE)
mouth_open_vec = get_vector_from_label('Mouth_Slightly_Open', BATCH_SIZE)
smiling_vec = get_vector_from_label('Smiling', BATCH_SIZE)
lipstick_vec = get_vector_from_label('Wearing_Lipstick', BATCH_SIZE)
young_vec = get_vector_from_label('High_Cheekbones', BATCH_SIZE)
male_vec = get_vector_from_label('Male', BATCH_SIZE)
eyeglasses_vec = get_vector_from_label('Eyeglasses', BATCH_SIZE)
blonde_vec = get_vector_from_label('Blond_Hair', BATCH_SIZE)

In [ ]:
print('Smiling Vector')
add_vector_to_images(smiling_vec)

In [ ]:
print('Blond Vector')
add_vector_to_images(blonde_vec)

In [ ]:
print('Male Vector')
add_vector_to_images(male_vec)

In [ ]:
print('Eyeglasses Vector')
add_vector_to_images(eyeglasses_vec)

## 面部变形

In [ ]:

def morph_faces(start_image_file, end_image_file):

    factors = np.arange(0, 1, 0.1)

    att_specific = att[att['image_id'].isin([start_image_file, end_image_file])]
    att_specific = att_specific.reset_index()
    data_flow_label = image_loader.build(att_specific, 2)

    example_batch = next(data_flow_label)
    example_images = example_batch[0]

    z_points = vae.encoder.predict(example_images)


    fig = plt.figure(figsize=(18, 8))

    counter = 1

    img = example_images[0].squeeze()
    sub = fig.add_subplot(1, len(factors)+2, counter)
    sub.axis('off')        
    sub.imshow(img)

    counter+=1
    for factor in factors:
        changed_z_point = z_points[0] * (1-factor) + z_points[1]  * factor
        changed_image = vae.decoder.predict(np.array([changed_z_point]))[0]

        img = changed_image.squeeze()
        sub = fig.add_subplot(1, len(factors)+2, counter)
        sub.axis('off')
        sub.imshow(img)

        counter += 1

    img = example_images[1].squeeze()
    sub = fig.add_subplot(1, len(factors)+2, counter)
    sub.axis('off')        
    sub.imshow(img)

    plt.show()